# Лабораторная работа №5. Градиентный бустинг

**Цель:** Исследование Gradient Boosting и сравнение всех методов.


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

mush = pd.read_csv("data/mushrooms.csv")
cars = pd.read_csv("data/car_price.csv")

X_mush = mush.drop("class", axis=1)
y_mush = mush["class"]

X_cars = cars.drop("Price", axis=1)
y_cars = cars["Price"]

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_mush, y_mush, test_size=0.2, random_state=42, stratify=y_mush
)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cars, y_cars, test_size=0.2, random_state=42
)

preprocess_mush = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), X_mush.columns)
])

preprocess_cars = ColumnTransformer([
    ("num", StandardScaler(), X_cars.select_dtypes(include=["int64", "float64"]).columns),
    ("cat", OneHotEncoder(handle_unknown="ignore"), X_cars.select_dtypes(include=["object"]).columns)
])


In [5]:
gb_clf_baseline = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", GradientBoostingClassifier(random_state=42))
])

gb_clf_baseline.fit(X_train_m, y_train_m)
y_pred_m_gb = gb_clf_baseline.predict(X_test_m)

print("=== Gradient Boosting Baseline (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_gb))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_gb))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_gb))


=== Gradient Boosting Baseline (Mushrooms) ===
Accuracy: 0.9987692307692307

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  2 781]]


In [4]:
gb_reg_baseline = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", GradientBoostingRegressor(random_state=42))
])

gb_reg_baseline.fit(X_train_c, y_train_c)
y_pred_c_gb = gb_reg_baseline.predict(X_test_c)

mae_gb = mean_absolute_error(y_test_c, y_pred_c_gb)
rmse_gb = mean_squared_error(y_test_c, y_pred_c_gb)**0.5
r2_gb = r2_score(y_test_c, y_pred_c_gb)

print("=== Gradient Boosting Baseline (Cars) ===")
print("MAE:", mae_gb)
print("RMSE:", rmse_gb)
print("R^2:", r2_gb)


=== Gradient Boosting Baseline (Cars) ===
MAE: 1858.7583717027283
RMSE: 2300.025147056761
R^2: 0.8066950509063259


## 3. Анализ baseline градиентного бустинга

### 3.1. Классификация (Mushrooms) — GradientBoostingClassifier

Результаты:

- **Accuracy ≈ 0.99877**
- F1-score = 1.0 для обоих классов
- 2 ошибки на классе `p`

Сравнение с предыдущими алгоритмами:

| Модель                      | Accuracy |
|-----------------------------|----------|
| Logistic Regression         | 1.00000  |
| KNN                         | 1.00000  |
| Decision Tree               | 1.00000  |
| Random Forest               | 1.00000  |
| **Gradient Boosting**      | **0.99877** |

Бустинг здесь слегка уступает дереву и лесу, но это нормально:  
модели бустинга плохо переносят идеально разделимые наборы после One-Hot кодирования и иногда переобучаются на отдельных признаках.

**Вывод:** качество отличное, но избыточное для такой задачи.

---

### 3.2. Регрессия (Car Price Prediction) — GradientBoostingRegressor

Результаты:

- **MAE ≈ 1858.76**
- **RMSE ≈ 2300.02**
- **R² ≈ 0.80670**

Сравнение baseline:

| Модель                        | MAE     | RMSE    | R²      |
|-------------------------------|---------|---------|---------|
| Linear Regression             | 1810    | 2237    | 0.817   |
| Ridge Regression              | 1808    | 2233    | 0.818   |
| Decision Tree (improved)      | 2090    | 2633    | 0.747   |
| Random Forest (improved)      | 1871    | 2310    | 0.805   |
| **Gradient Boosting**         | **1859**| **2300**| **0.807**|

Градиентный бустинг:

- работает лучше случайного леса,
- лучше решающих деревьев,
- почти догоняет линейные модели,
- показывает одно из лучших baseline-качеств среди всех алгоритмов.

**Вывод:** бустинг — сильная модель для задачи регрессии даже без настройки гиперпараметров.


In [6]:
gb_clf_pipe = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", GradientBoostingClassifier(random_state=42))
])

param_grid_clf = {
    "model__n_estimators": [100, 200],
    "model__learning_rate": [0.05, 0.1, 0.2],
    "model__max_depth": [2, 3, 5]
}

grid_gb_clf = GridSearchCV(
    gb_clf_pipe,
    param_grid_clf,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

grid_gb_clf.fit(X_train_m, y_train_m)

print("Best params (GB Mushrooms):", grid_gb_clf.best_params_)

best_gb_clf = grid_gb_clf.best_estimator_
y_pred_m_gb_best = best_gb_clf.predict(X_test_m)

print("\n=== Improved Gradient Boosting (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_gb_best))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_gb_best))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_gb_best))


Best params (GB Mushrooms): {'model__learning_rate': 0.05, 'model__max_depth': 3, 'model__n_estimators': 100}

=== Improved Gradient Boosting (Mushrooms) ===
Accuracy: 0.9987692307692307

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  2 781]]


In [7]:
gb_reg_pipe = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", GradientBoostingRegressor(random_state=42))
])

param_grid_reg = {
    "model__n_estimators": [100, 200],
    "model__learning_rate": [0.05, 0.1, 0.2],
    "model__max_depth": [2, 3, 5]
}

grid_gb_reg = GridSearchCV(
    gb_reg_pipe,
    param_grid_reg,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

grid_gb_reg.fit(X_train_c, y_train_c)

print("Best params (GB Cars):", grid_gb_reg.best_params_)

best_gb_reg = grid_gb_reg.best_estimator_
y_pred_c_gb_best = best_gb_reg.predict(X_test_c)

mae_gb_best = mean_absolute_error(y_test_c, y_pred_c_gb_best)
rmse_gb_best = mean_squared_error(y_test_c, y_pred_c_gb_best)**0.5
r2_gb_best = r2_score(y_test_c, y_pred_c_gb_best)

print("\n=== Improved Gradient Boosting (Cars) ===")
print("MAE:", mae_gb_best)
print("RMSE:", rmse_gb_best)
print("R^2:", r2_gb_best)


Best params (GB Cars): {'model__learning_rate': 0.05, 'model__max_depth': 2, 'model__n_estimators': 200}

=== Improved Gradient Boosting (Cars) ===
MAE: 1860.7825456755863
RMSE: 2309.7731792322825
R^2: 0.805053037553818


## 3. Улучшение (GridSearchCV)

1. **Mushrooms**: Accuracy = 0.999 (без изменений).
2. **Cars**: Гиперпараметры почти не влияют на качество (R2 ~0.805). Модель стабильна.


# 4. Итоговое сравнение всех алгоритмов

| Алгоритм | Задача | Лучший R2 / Acc |
|---|---|---|
| **KNN** | Reg | 0.734 |
| **Linear Regression** | Reg | **0.817** |
| **Ridge** | Reg | **0.818** |
| **Decision Tree** | Reg | 0.747 |
| **Random Forest** | Reg | 0.805 |
| **Gradient Boosting** | Reg | 0.807 |

**Выводы:**
1. Для задачи **классификации** (грибы) все методы показали идеальный результат (Accuracy ~1.0).
2. Для задачи **регрессии** (автомобили) лучшими оказались **линейные модели** (Ridge) и **ансамбли** (Forest, Boosting).
3. Простые методы (KNN, одиночное дерево) значительно уступают ансамблям и линейным моделям на данной задаче.
4. Все алгоритмы были успешно реализованы и протестированы.
